In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!git clone https://github.com/valkryhx/localGPT

fatal: destination path 'localGPT' already exists and is not an empty directory.


In [3]:
%cd localGPT/
!pip install -r requirements.txt

/kaggle/working/localGPT
Ignoring protobuf: markers 'sys_platform == "darwin" and platform_machine != "arm64"' don't match your environment
Ignoring protobuf: markers 'sys_platform == "darwin" and platform_machine == "arm64"' don't match your environment
Ignoring bitsandbytes-windows: markers 'sys_platform == "win32"' don't match your environment
  Using cached urllib3-1.26.6-py2.py3-none-any.whl (138 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.16
    Uninstalling urllib3-1.26.16:
      Successfully uninstalled urllib3-1.26.16
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2023.3.2.1 requires dask==2023.3.2, but you have dask 2023.7.0 which is incompatible.
google-cloud-datastore 2.16.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have prot

In [4]:
cat /kaggle/working/localGPT/constants.py

import os

# from dotenv import load_dotenv
from chromadb.config import Settings

# https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/excel.html?highlight=xlsx#microsoft-excel
from langchain.document_loaders import CSVLoader, PDFMinerLoader, TextLoader, UnstructuredExcelLoader, Docx2txtLoader

# load_dotenv()
ROOT_DIRECTORY = os.path.dirname(os.path.realpath(__file__))

# Define the folder for storing database
SOURCE_DIRECTORY = f"{ROOT_DIRECTORY}/SOURCE_DOCUMENTS"

PERSIST_DIRECTORY = f"{ROOT_DIRECTORY}/DB"

# Can be changed to a specific number
INGEST_THREADS = os.cpu_count() or 8

# Define the Chroma settings
CHROMA_SETTINGS = Settings(
    chroma_db_impl="duckdb+parquet", persist_directory=PERSIST_DIRECTORY, anonymized_telemetry=False,
    collection_metadata={"hnsw:space": "cosine"}  ## add 20230831
)

# https://python.langchain.com/en/latest/_modules/langchain/document_loaders/excel.html#UnstructuredExcelLoader
DOCUMENT_MAP = {
    ".txt": TextLoade

In [5]:
!git pull --all --force
!python ingest.py

Fetching origin
Already up to date.
Traceback (most recent call last):
  File "/kaggle/working/localGPT/ingest.py", line 12, in <module>
    from constants import (
  File "/kaggle/working/localGPT/constants.py", line 21, in <module>
    CHROMA_SETTINGS = Settings(
  File "pydantic/env_settings.py", line 40, in pydantic.env_settings.BaseSettings.__init__
  File "pydantic/main.py", line 341, in pydantic.main.BaseModel.__init__
pydantic.error_wrappers.ValidationError: 1 validation error for Settings
collection_metadata
  extra fields not permitted (type=value_error.extra)


# 上面这个max_se_length 其实是https://github.com/langchain-ai/langchain/blob/0689628489967785f3a11a9f29d8f6f90930f4f4/libs/langchain/langchain/embeddings/huggingface.py#L231C9-L231C65
Breadcrumbslangchain/libs/langchain/langchain/embeddings
/huggingface.py 231行sentence_transformers.SentenceTransformer 加载SentenceTransformer 默认的512  要动的话需要动源码

ls DB/  ingest 之后的向量文件在这里  如果不想要就 rm -rf DB 那下次就要重新ingest

In [ ]:
#run_localGPT.py 中的query=input()无法在kaggle环境上正常执行 我们自己把代码拿出来跑
#!python run_localGPT.py

In [6]:
# codes are just from local_GPT.py
import logging

import click
import torch
from auto_gptq import AutoGPTQForCausalLM
from huggingface_hub import hf_hub_download
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.llms import HuggingFacePipeline, LlamaCpp
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from loguru import logger
# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.vectorstores import Chroma
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    GenerationConfig,
    LlamaForCausalLM,
    LlamaTokenizer,
    pipeline,
)

from constants import CHROMA_SETTINGS, EMBEDDING_MODEL_NAME, PERSIST_DIRECTORY, MODEL_ID, MODEL_BASENAME

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:25                                                                                   │
│                                                                                                  │
│   22 │   pipeline,                                                                               │
│   23 )                                                                                           │
│   24                                                                                             │
│ ❱ 25 from constants import CHROMA_SETTINGS, EMBEDDING_MODEL_NAME, PERSIST_DIRECTORY, MODEL_ID    │
│   26                                                                                             │
│                                                                                                  │
│ /kaggle/working/localGPT/constants.py:21 in <module>                                             │
│                                                                                                  │
│   18 INGEST_THREADS = os.cpu_count() or 8                                                        │
│   19                                                                                             │
│   20 # Define the Chroma settings                                                                │
│ ❱ 21 CHROMA_SETTINGS = Settings(                                                                 │
│   22 │   chroma_db_impl="duckdb+parquet", persist_directory=PERSIST_DIRECTORY, anonymized_tel    │
│   23 │   collection_metadata={"hnsw:space": "cosine"}  ## add 20230831                           │
│   24 )                                                                                           │
│                                                                                                  │
│ in pydantic.env_settings.BaseSettings.__init__:40                                                │
│                                                                                                  │
│ in pydantic.main.BaseModel.__init__:341                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValidationError: 1 validation error for Settings
collection_metadata
  extra fields not permitted (type=value_error.extra)

In [ ]:
# 这里会下载和加载chatglm2-6b
!git pull --all --force
from my_chatglm_llm import ChatGLM

In [ ]:
h=[("123"),("3")]
sum([len(item) for item in h])

In [ ]:
!git pull --all --force

In [ ]:

device_type="cuda"
show_sources =True
EMBEDDING_MODEL_NAME = 'moka-ai/m3e-base'
logging.info(f"Running on: {device_type}")
logging.info(f"Display Source Documents set to: {show_sources}")
logger.error(f"EMBEDDING_MODEL_NAME={EMBEDDING_MODEL_NAME}")
embeddings = HuggingFaceInstructEmbeddings(model_name=EMBEDDING_MODEL_NAME, model_kwargs={"device": device_type})

# uncomment the following line if you used HuggingFaceEmbeddings in the ingest.py
# embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)

# load the vectorstore
db = Chroma(
        persist_directory=PERSIST_DIRECTORY,
        embedding_function=embeddings,
        client_settings=CHROMA_SETTINGS,
    )
#retriever = db.as_retriever()

retriever = db.as_retriever(
    search_type="similarity_score_threshold", 
    search_kwargs={"k":3, "score_threshold":0.75}
    )

In [ ]:
template = """使用如下信息回答问题. 如果不知道答案,\
    就回答不知道，不要编造答案.

    {context}

    {history}
    问: {question}
    答:"""

prompt = PromptTemplate(input_variables=["history", "context", "question"], template=template)
memory = ConversationBufferMemory(input_key="question", memory_key="history")
logger.error(isinstance(memory,list)) # memory 不是 list
#print(memory)
#llm = load_model(device_type, model_id=MODEL_ID, model_basename=MODEL_BASENAME)
llm = ChatGLM()
qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt, "memory": memory},
    )
    # Interactive questions and answers
cnt = 0

In [ ]:
# 注意  打印history的语句在chatglm_llm.py最下方 那里也可以调节history的取值 我现在取值history[-2:0]
while cnt == 0:
        #cnt += 1
        query = input("输入问题:\n")
        #query = "how long is the period of united states president?"
        #query = "SSE报销预览打印错误怎么办"
        if query == "exit":
            break
        # Get the answer from the chain
        res = qa(query)
        print("12345")
        answer, docs = res["result"], res["source_documents"]

        # Print the result
        print("\n\n> Question:")
        print(query)
        print("\n> Answer:")
        print(answer)

        if show_sources:  # this is a flag that you can set to disable showing answers.
            # # Print the relevant sources used for the answer
            print("----------------------------------SOURCE DOCUMENTS---------------------------")
            for document in docs:
                print("\n> " + document.metadata["source"] + ":")
                print(document.page_content)
            print("----------------------------------SOURCE DOCUMENTS---------------------------")

In [ ]:
https://github.com/langchain-ai/langchain/issues/6481
https://docs.trychroma.com/usage-guide#changing-the-distance-function
https://stackoverflow.com/questions/76678783/langchains-chroma-vectordb-similarity-search-with-score-and-vectordb-simil
https://github.com/langchain-ai/langchain/issues/5458  
https://github.com/langchain-ai/langchain/blob/e60e1cdf23ad73b2e0a40034c0ddfc3c8b0c9c4d/libs/langchain/langchain/vectorstores/base.py#L460